In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
from scipy.stats import norm
import time as ttt
import iisignature as iisig
from tqdm import *
from einops import rearrange

In [2]:
import torch
import torch.nn as nn
import pandas as pd 

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

data_type=torch.float32

In [13]:
x0 = 100.0 # initial condition
sigma = 0.15 # volatility
d=20
K=100.0

segs=20
r = 0.05 # risk free rate
batch_size = 100 # batch size
steps=1000
T = 1 # maturity
dt = T/steps # mesh size

sqrt_dt=np.sqrt(dt); 
dt_new = T/segs # new mesh after shrinkage
level = 3 # truncation level logsig 91 sig 258

In [14]:
batch_in=100
MOMENTUM = 0.99
EPSILON = 1e-6
import warnings
warnings.filterwarnings("ignore")

In [15]:
def create_stock(x0,r,sigma,T,steps,n_path,dW):  
    s_vec=[];
    w0=np.ones((n_path,1,d))*1e-6
    dW=np.concatenate((w0,dW),axis=1) ## the first time slot is then 0
    s_vec.append(np.ones((n_path,d))*100.0)
    for i in range(steps): 
        s_vectemp=s_vec[-1]+ r*s_vec[-1]*dt+ dW[:,i+1,:]*s_vec[-1]*sigma
        s_vec.append(s_vectemp)
    BM_path=np.cumsum(dW,axis=1) ## find the cumulative sum
    S_path=rearrange(np.array(s_vec), 'b c h -> c b h') 
    return BM_path, S_path

def jointime(T,path): 
    n_path, steps, d=path.shape
    dt=T/(steps-1); 
    
    times=np.arange(0,T,dt) ## This can be taken out
    times=np.append(times,T); 
    times_vec=np.tile(times,[1,1]); 
    times_vec=np.transpose(times_vec)
    times_vec=np.tile(times_vec,[n_path,1,1])
    times_vec=np.concatenate((times_vec, path),axis=2)
    return times_vec

def ComputeMultiLevelSig(path, number_of_segment, depth,log_sig=False):
    n_batch, nsteps, chanels = path.shape
    t_vec = np.arange(0, nsteps-1, int(nsteps / number_of_segment))
    t_vec = np.append(t_vec, nsteps-1)
    MultiLevelSig = []
    s=iisig.prepare(d+1,depth)
    
    if log_sig: 
        ll=iisig.logsig(np.expand_dims(path[:,0,:],axis=1),s)
        MultiLevelSig.append(ll)
        for i in range(len(t_vec)-1):    
        ## Notice that we only use the signature of the concatenation of time and space.
            MultiLevelSig.append(iisig.logsig(path[:,0:t_vec[i+1]+1,:],s)) ##if not
        MultiLevelSig=np.stack(MultiLevelSig)  
        MultiLevelSig=rearrange(MultiLevelSig, 'b c h -> c b h') 
    else: 
        ll=iisig.sig(np.expand_dims(path[:,0,:],axis=1),depth)
        MultiLevelSig.append(ll)
        for i in range(len(t_vec)-1):    
        ## Notice that we only use the signature of the concatenation of time and space.
            MultiLevelSig.append(iisig.sig(path[:,0:t_vec[i+1]+1,:],depth)) ##if not
            #MultiLevelSig.append(path_class.signature(t_vec[i],t_vec[i+1]+1))
        MultiLevelSig=np.stack(MultiLevelSig)  
        MultiLevelSig=rearrange(MultiLevelSig, 'b c h -> c b h') 
        
    return MultiLevelSig

In [20]:
dW=np.sqrt(dt)*np.random.normal(size=(100, steps,d))
bm_p,s_path=create_stock(x0,r,sigma,T,steps,100,dW)
tv=jointime(T,bm_p)
ms=ComputeMultiLevelSig(tv, segs, 3,log_sig=True)

In [21]:
ms.shape

(100, 21, 3311)

In [22]:
### The x_ten here has to be a path
### The level of discretization is on the fine mesh
def terminal_loss(x_ten):
    res=torch.relu(torch.sum(x_ten,dim=(1,2))*dt/d-K)
    return res

In [24]:
def generate_samples(batch_in=100):
    """
    Produce the signature, dW for computation, YT the terminal condition
    x_ten, selection the path and location to be used
    """
    dW=np.sqrt(dt)*np.random.normal(size=(batch_in, steps,d))
    
    pth2=create_stock(x0,r,sigma,T,steps,batch_in,dW)
    BM_timePath=jointime(T,pth2[0]); 
    S_timePath=jointime(T,pth2[1]);
    
    sigs=ComputeMultiLevelSig(S_timePath, 20, 3)
    selection = np.linspace(0,steps, segs+1, dtype = np.int)

    BM_seg=BM_timePath[:,selection,1:]
    dW=BM_seg[:,1:,:]-BM_seg[:,:-1,:]

    dW=torch.tensor(dW,dtype=torch.float32)
    sigs=torch.tensor(sigs,dtype=torch.float32)
    S_timePath=torch.tensor(S_timePath,dtype=torch.float32)
    
    x_ten=S_timePath[:,:,1:]
    
    YT=torch.relu(torch.sum(x_ten,dim=(1,2))*dt/d-K)
    
    return sigs, dW, YT, x_ten, selection[:-1]

In [25]:
class Config(object):
    n_layer = 4
    batch_size = 1024
    valid_size = 1024
    
    dim=9723; 
    Ntime=20; 
    delta=1/Ntime
    sqrt_deltaT=np.sqrt(1.0/Ntime); 
    lam=1; 

    logging_frequency = 100
    verbose = True
    y_init_range = [0, 1]
    
    num_hiddens = [dim,64,64,d] ## 256 ,256
    
def get_config(name):
    try:
        return globals()[name]
    except KeyError:
        raise KeyError("config not defined.")

cfg=get_config('Config')

In [26]:
class Dense(nn.Module): 
    def __init__(self,cin, cout, batch_norm=False, activate=True): 
        super(Dense,self).__init__()
        self.cin=cin; 
        self.cout=cout; 
        self.activate=activate; 
        
        self.linear=nn.Linear(self.cin,self.cout) #The linear layer
        #BatchNorm1d: it requires the input to be a correct size
        if batch_norm: 
            self.bn=nn.BatchNorm1d(cout,eps=EPSILON,momentum=MOMENTUM)
        else: 
            self.bn=None
      #  nn.init.normal_(self.linear.weight,std=5.0/np.sqrt(cin+cout))
        # This is the He initialization
        
    def forward(self,x): 
        x=self.linear(x)
        if self.bn is not None:
            x=self.bn(x)
        if self.activate:
            x=torch.tanh(x)
        return x 
    
class FFN(nn.Module):
    def __init__(self, config):
        super(FFN,self).__init__()
        self.config=config
        
        self.bn=nn.BatchNorm1d(config.num_hiddens[0],eps=EPSILON,momentum=MOMENTUM) ## So there is batch norm no problem
        # range(1,5): 1,2,3,4
        self.layers=[Dense(config.num_hiddens[i-1],config.num_hiddens[i]) for i in range(1, len(config.num_hiddens)-1)]
        self.layers+=[Dense(config.num_hiddens[-2], config.num_hiddens[-1],activate=False)]
        self.layers=nn.Sequential(*self.layers)
    
    def forward(self,x):
      #  x=self.bn(x)
        x=self.layers(x)
        return x 
    
class Lookback_PPDE_Backward(nn.Module):
    def __init__(self,cfg): 
        super(Lookback_PPDE_Backward,self).__init__()
        self.cfg=cfg
        self.Ntime=self.cfg.Ntime 
        self.mList=nn.ModuleList([FFN(self.cfg) for _ in range(self.Ntime)])
        
    def forward(self,batch_sig,batch_dW,batch_YT, batch_x, batch_sel): 
        Y=batch_YT
        for i in np.arange(segs-1,-1, -1):
            Y=Y-Y*r*T/segs-sigma*torch.sum( self.mList[i](batch_sig[:,i,:])*batch_dW[:,i,:],axis=1,keepdim=True)
            gi=terminal_loss(batch_x[:,:batch_sel[i],:])
            Y=torch.relu(gi-Y)+Y
        return Y

In [27]:
def loss_var(x):
    temp=torch.var(x)
    return temp

In [38]:
import torch.optim as optim
from torch.nn import Parameter
import math
model_PPDE_bw=Lookback_PPDE_Backward(cfg)
model_PPDE_bw
optimizer=optim.Adam(model_PPDE_bw.parameters(),lr=5e-4)
grad_clip=0.1
y0_mean=[];
loss_vec=[];
#scheduler=torch.optim.lr_scheduler.StepLR(optimizer, step_size=80, gamma=0.1)

## 0.5823 -- 0.5784 -- 0.5862

In [ ]:
for i in range(50):
    batch_sig, batch_dw, batch_y,batch_x, batch_sel =generate_samples(batch_in=500)

    x_temp=model_PPDE_bw(batch_sig,batch_dw,batch_y,batch_x, batch_sel)
    loss_temp=loss_var(x_temp)

    optimizer.zero_grad()
    loss_temp.backward()
    optimizer.step()
#   scheduler.step()
    y0_val=x_temp.mean().cpu().detach().numpy()
    loss_val=loss_temp.cpu().detach().numpy()
    
    y0_mean.append(y0_val)
    loss_vec.append(loss_val)
    if grad_clip: 
        nn.utils.clip_grad_value_(model_PPDE_bw.parameters(), grad_clip)
    
    print("Iter:", i, 'The mean Y0 is', y0_val , 'Variance is:' ,loss_val)

Iter: 0 The mean Y0 is 2.675419 Variance is: 2.9787557
Iter: 1 The mean Y0 is 2.7662926 Variance is: 3.3489914
Iter: 2 The mean Y0 is 2.6716583 Variance is: 3.1696858
Iter: 3 The mean Y0 is 2.5366142 Variance is: 3.0067346
Iter: 4 The mean Y0 is 2.644172 Variance is: 2.9511125


In [ ]:
df=pd.DataFrame()
df['y_pred']=y0_mean
df['loss_var']=loss_vec

df_temp=df[11:]
lw=df_temp.loss_var.mean()-2.0*df_temp.loss_var.std()
up=df_temp.loss_var.mean()+2.0*df_temp.loss_var.std()
df1=df_temp[df_temp.loss_var>=lw]
df1=df1[df1.loss_var<=up]
df1.y_pred.mean()

In [36]:
lw

2.465573030665355

In [37]:
up

3.661809687934405

In [ ]:
for epoch in np.arange(31,51,1):
    model_PPDE_bw=Lookback_PPDE_Backward(cfg)
    optimizer=optim.Adam(model_PPDE_bw.parameters(),lr=5e-4)
    grad_clip=0.1
    y0_mean=[];
    loss_vec=[];
    for i in range(40):
        batch_sig, batch_dw, batch_y,batch_x, batch_sel =generate_samples(batch_in=500)

        x_temp=model_PPDE_bw(batch_sig,batch_dw,batch_y,batch_x, batch_sel)
        loss_temp=loss_var(x_temp)

        optimizer.zero_grad()
        loss_temp.backward()
        optimizer.step()
    #   scheduler.step()
        y0_val=x_temp.mean().cpu().detach().numpy()
        loss_val=loss_temp.cpu().detach().numpy()

        y0_mean.append(y0_val)
        loss_vec.append(loss_val)
        if grad_clip: 
            nn.utils.clip_grad_value_(model_PPDE_bw.parameters(), grad_clip)

        print("Iter:", i, 'The mean Y0 is', y0_val , 'Variance is:' ,loss_val)
    
    print("Epoch:", epoch)
    
    y_pred=np.array(y0_mean)
    loss_val=np.array(loss_vec)
    iters=np.arange(0,len(y_pred),1)+1
    df=pd.DataFrame()
    df['iter']=iters
    df['y_pred']=y_pred
    df['loss_var']=loss_val
    df.to_csv('eg2_trained_dataDim10/method2'+'_'+str(epoch)+'.csv')

Iter: 0 The mean Y0 is 2.8294892 Variance is: 5.50975
Iter: 1 The mean Y0 is 2.8102558 Variance is: 5.3654604
Iter: 2 The mean Y0 is 2.8236651 Variance is: 5.747853
Iter: 3 The mean Y0 is 2.632239 Variance is: 4.796325
Iter: 4 The mean Y0 is 2.6199396 Variance is: 4.9421134
Iter: 5 The mean Y0 is 2.8141148 Variance is: 5.2012453
Iter: 6 The mean Y0 is 2.6397488 Variance is: 5.090225
Iter: 7 The mean Y0 is 2.7784877 Variance is: 5.434707
Iter: 8 The mean Y0 is 2.8528817 Variance is: 5.2322693
Iter: 9 The mean Y0 is 2.778217 Variance is: 5.6350026
Iter: 10 The mean Y0 is 2.831774 Variance is: 5.810942
Iter: 11 The mean Y0 is 2.6345754 Variance is: 5.2025023
Iter: 12 The mean Y0 is 2.8138704 Variance is: 5.3736877
Iter: 13 The mean Y0 is 2.827101 Variance is: 5.38329
Iter: 14 The mean Y0 is 2.65377 Variance is: 4.6350174
Iter: 15 The mean Y0 is 2.7636387 Variance is: 5.181613
Iter: 16 The mean Y0 is 2.649039 Variance is: 5.413356
Iter: 17 The mean Y0 is 2.920896 Variance is: 5.1981864
Ite

Iter: 27 The mean Y0 is 2.5863085 Variance is: 5.2850637
Iter: 28 The mean Y0 is 2.7842965 Variance is: 5.5077786
Iter: 29 The mean Y0 is 2.6088662 Variance is: 5.0743027
Iter: 30 The mean Y0 is 2.73263 Variance is: 5.664233
Iter: 31 The mean Y0 is 2.7073185 Variance is: 5.503865
Iter: 32 The mean Y0 is 2.7641284 Variance is: 5.2170777
Iter: 33 The mean Y0 is 2.763131 Variance is: 5.292246
Iter: 34 The mean Y0 is 2.7867868 Variance is: 5.624924
Iter: 35 The mean Y0 is 2.8001688 Variance is: 5.225587
Iter: 36 The mean Y0 is 2.787465 Variance is: 5.115392
Iter: 37 The mean Y0 is 2.7480595 Variance is: 5.085086
Iter: 38 The mean Y0 is 2.903711 Variance is: 5.0783553
Iter: 39 The mean Y0 is 2.9774964 Variance is: 5.0607996
Epoch: 34
Iter: 0 The mean Y0 is 2.672758 Variance is: 5.14066
Iter: 1 The mean Y0 is 2.4208357 Variance is: 4.406251
Iter: 2 The mean Y0 is 2.7474027 Variance is: 5.6633625
Iter: 3 The mean Y0 is 2.755661 Variance is: 5.415977
Iter: 4 The mean Y0 is 2.7500937 Variance i

Iter: 13 The mean Y0 is 2.5327773 Variance is: 5.1878743
Iter: 14 The mean Y0 is 2.6629205 Variance is: 5.5148287
Iter: 15 The mean Y0 is 2.955307 Variance is: 6.199605
Iter: 16 The mean Y0 is 2.8266597 Variance is: 5.54546
Iter: 17 The mean Y0 is 2.963856 Variance is: 5.6018786
Iter: 18 The mean Y0 is 2.7714782 Variance is: 5.051026
Iter: 19 The mean Y0 is 2.820866 Variance is: 5.289503
Iter: 20 The mean Y0 is 2.715976 Variance is: 5.2211394
Iter: 21 The mean Y0 is 2.6907794 Variance is: 5.465856
Iter: 22 The mean Y0 is 2.8761573 Variance is: 5.105583
Iter: 23 The mean Y0 is 2.727224 Variance is: 5.4933977
Iter: 24 The mean Y0 is 2.6719658 Variance is: 5.069692
Iter: 25 The mean Y0 is 2.73042 Variance is: 5.000891
Iter: 26 The mean Y0 is 2.6792917 Variance is: 5.2663903
Iter: 27 The mean Y0 is 2.925559 Variance is: 5.564169
Iter: 28 The mean Y0 is 2.721246 Variance is: 5.819167
Iter: 29 The mean Y0 is 2.8746793 Variance is: 5.159397
Iter: 30 The mean Y0 is 2.9793172 Variance is: 5.307

Iter: 0 The mean Y0 is 2.783816 Variance is: 4.9907722
Iter: 1 The mean Y0 is 2.727931 Variance is: 5.057173
Iter: 2 The mean Y0 is 2.8317142 Variance is: 5.455662
Iter: 3 The mean Y0 is 2.7482917 Variance is: 5.1119394
Iter: 4 The mean Y0 is 2.8163028 Variance is: 5.3652153
Iter: 5 The mean Y0 is 2.8991425 Variance is: 5.450378
Iter: 6 The mean Y0 is 2.728086 Variance is: 5.418075
Iter: 7 The mean Y0 is 2.815375 Variance is: 5.413872
Iter: 8 The mean Y0 is 2.8899062 Variance is: 5.5549064
Iter: 9 The mean Y0 is 2.8289845 Variance is: 5.3736014
Iter: 10 The mean Y0 is 2.716765 Variance is: 5.1676083
Iter: 11 The mean Y0 is 2.7009218 Variance is: 5.27644
Iter: 12 The mean Y0 is 2.8463268 Variance is: 5.476079
Iter: 13 The mean Y0 is 2.9569924 Variance is: 5.660862
Iter: 14 The mean Y0 is 2.6711826 Variance is: 4.929688
Iter: 15 The mean Y0 is 2.750771 Variance is: 5.546639
Iter: 16 The mean Y0 is 2.886883 Variance is: 5.0224423
Iter: 17 The mean Y0 is 2.7751312 Variance is: 5.4135113
It

2.591178698417468